## Predicting academic performance using demographic and behavioral Data

by Zhengling Jiang,  Colombe Tolokin, Franklin Aryee, Tien Nguyen

Packages:

## Summary

## Introduction

## Methods

### Data

### Analysis

## Results & Discussion

## References